In [2]:
import SAP_GUI
import win32com.client
import pandas as pd
import subprocess
from time import sleep
from datetime import datetime
from datetime import timedelta


In [3]:
"""----------------------------Inciar session----------------------------------------------------"""
# Insert User name and password
Keys=pd.read_excel("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Claves\\Keys.xlsx")
user=Keys["User"][0]
password=Keys["Password"][0]
# Initialize session
session=SAP_GUI.SessionSAP(user,password)

In [10]:
transsaccion="ZSD035D"
variant="JESPINOSAP"
provision="INFORME FACTU"

def Search_ZSD035D(Transsaccion,session,provision,date,variant):
    session.StartTransaction(Transsaccion)
    session.findById("wnd[0]/tbar[1]/btn[17]").press()
    session.findById("wnd[1]/usr/txtV-LOW").text = provision
    session.findById("wnd[1]/usr/txtENAME-LOW").text = variant
    session.findById("wnd[1]/usr/txtV-LOW").caretPosition = 8
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    if date==None:
        None
    else:
        session.findById("wnd[0]/usr/ctxtSP$00024-LOW").text = date
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    try:
        table=session.findById("wnd[0]/usr/cntlCONTAINER/shellcont/shell")
        return(table)
    except:
        return(session)

In [11]:
Search_ZSD035D(transsaccion,session,provision,"13.06.2023",variant)

<COMObject <unknown>>

In [12]:
# Pendiente Firme
Name_ZSD035D="Pendiente_Firme"  # Revisar suele no guardar
SAP_GUI.Export_TXT2(Name_ZSD035D,session)
